# Internship 01 

# Client Requests
1. The operations team wants to ensure the integrity of the events data by removing
duplicates. Check for and remove duplicate rows in the events dataframe based
on store_id, campaign_id, and product_code. How many duplicate rows were
removed?

2. How many cities have more than 5 stores?

3. The sales team has identified missing values in the quantity_sold(before_promo)
data. Estimate these values using the median quantity sold before the promotion.
How many missing values were filled, and what is the median used for
imputation?
4. Identify the product category with the lowest base price before the promotion.
5. What is the total quantity sold after the promotion for the BOGOF promo type
during the Diwali campaign?
6. Which store recorded the highest quantity sold after the promotion during the
Diwali campaign?
7. Understand which campaigns had the most successful outcomes. Compare the
total quantities sold before and after the promotions for the Sankranti and Diwali
campaigns. Which campaign saw a greater increase in sales?
8. Which product recorded the highest Incremental Revenue Percentage (IR%)
during the Sankranti campaign? What is the IR% for this product?
9. Which store in Visakhapatnam recorded the lowest Incremental Sold Units
Percentage (ISU%) during the Diwali campaign? What is the ISU% for that
store?
10.Which promo type had both a negative Incremental Revenue Percentage (IR%)
and Incremental Sold Units Percentage (ISU%) during the Sankranti campaign?


Key Metrics:
 - IR% (Incremental Revenue): IR% measures the percentage change in revenue
after a promotion compared to the revenue before the promotion. It helps assess
how effective a promotion was in driving revenue growth.
 - ISU% (Incremental Sold Units): ISU% calculates the percentage change in the
number of units sold after a promotion compared to the units sold before the
promotion. It indicates the effectiveness of a promotion in boosting sales volume.

# Import All datasets

In [2]:
import pandas as pd
# Load the uploded CSV files
df_events = pd.read_csv("./datasets/fact_events.csv")
df_campaigns=pd.read_csv("./datasets/dim_campaigns.csv")
df_products=pd.read_csv("./datasets/dim_products.csv")
df_stores=pd.read_csv("./datasets/dim_stores.csv")

In [3]:
# Display the shapes of all datafrmes
{"df_events": df_events.shape,"df_campaigns":df_campaigns.shape, "df_products": df_products.shape, "df_stores":df_stores.shape}

{'df_events': (1510, 9),
 'df_campaigns': (2, 4),
 'df_products': (15, 3),
 'df_stores': (50, 2)}

In [4]:
df_events.head(2)

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo)
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327.0,25% OFF,217,287
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379.0,BOGOF,185,1622


In [5]:
df_campaigns.head(2)

,campaign_id,campaign_name,start_date,end_date
0,CAMP_DIW_01,Diwali,12-11-2023,18-11-2023
1,CAMP_SAN_01,Sankranti,10-01-2024,16-01-2024


In [6]:
df_products.head(2)

,product_code,product_name,category
0,P01,Atliq_Masoor_Dal (1KG),Grocery & Staples
1,P02,Atliq_Sonamasuri_Rice (10KG),Grocery & Staples


In [7]:
df_stores.head(2)

,store_id,city
0,STTRV-0,Trivandrum
1,STMDU-3,Madurai


In [8]:
# Merge on keys (example keys: campaign_id, product_id, store_id)       # \ (backslash) --> Used to improve code readability, especially with method chaining or long statements
merged = df_events \
    .merge(df_campaigns, on='campaign_id', how='left') \
    .merge(df_products, on='product_code', how='left') \
    .merge(df_stores, on='store_id', how='left')

# Alternate to the above 
""" can be used in single line
merged = (
    df_events
    .merge(df_campaigns,on="campaign_id",how="left")
    .merge(df_products,on="product_code", how="left")
    .merge(df_stores, on="store_id", how="left")
)
"""
# Preview result
merged.head()

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),campaign_name,start_date,end_date,product_name,category,city
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327.0,25% OFF,217,287,Diwali,12-11-2023,18-11-2023,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,Chennai
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379.0,BOGOF,185,1622,Sankranti,10-01-2024,16-01-2024,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,Chennai
2,f30579,STBLR-9,CAMP_DIW_01,P02,860,337.0,33% OFF,576,488,Diwali,12-11-2023,18-11-2023,Atliq_Sonamasuri_Rice (10KG),Grocery & Staples,Bengaluru
3,4f570c,STBLR-7,CAMP_DIW_01,P05,55,122.0,25% OFF,41,107,Diwali,12-11-2023,18-11-2023,Atliq_Scrub_Sponge_For_Dishwash,Home Care,Bengaluru
4,6d153f,STHYD-5,CAMP_SAN_01,P15,3000,122.0,500 Cashback,2500,272,Sankranti,10-01-2024,16-01-2024,Atliq_Home_Essential_8_Product_Combo,Combo1,Hyderabad


In [9]:
merged.shape  # No alteration in rows , but columns added are 6. 

(1510, 15)

### **1.** The operations team wants to ensure the integrity of the events data by removing duplicates. Check for and remove duplicate rows in the events dataframe based on **store_id, campaign_id, and product_code**. How many duplicate rows were removed?

In [10]:
merged.duplicated().sum()

10

In [11]:
merged[merged.duplicated(keep=False)]

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),campaign_name,start_date,end_date,product_name,category,city
40,46d57b,STBLR-9,CAMP_DIW_01,P13,350,94.0,BOGOF,175,329,Diwali,12-11-2023,18-11-2023,Atliq_High_Glo_15W_LED_Bulb,Home Appliances,Bengaluru
74,46d57b,STBLR-9,CAMP_DIW_01,P13,350,94.0,BOGOF,175,329,Diwali,12-11-2023,18-11-2023,Atliq_High_Glo_15W_LED_Bulb,Home Appliances,Bengaluru
126,4ad12b,STBLR-4,CAMP_DIW_01,P15,3000,407.0,500 Cashback,2500,1245,Diwali,12-11-2023,18-11-2023,Atliq_Home_Essential_8_Product_Combo,Combo1,Bengaluru
136,f6aa36,STVSK-3,CAMP_DIW_01,P06,415,63.0,25% OFF,311,54,Diwali,12-11-2023,18-11-2023,Atliq_Fusion_Container_Set_of_3,Home Care,Visakhapatnam
333,0491f4,STVSK-2,CAMP_DIW_01,P11,190,38.0,50% OFF,95,58,Diwali,12-11-2023,18-11-2023,Atliq_Doodh_Kesar_Body_Lotion (200ML),Personal Care,Visakhapatnam
395,6.24E+11,STMLR-0,CAMP_SAN_01,P03,200,206.0,BOGOF,100,541,Sankranti,10-01-2024,16-01-2024,Atliq_Suflower_Oil (1L),Grocery & Staples,Mangalore
410,f6aa36,STVSK-3,CAMP_DIW_01,P06,415,63.0,25% OFF,311,54,Diwali,12-11-2023,18-11-2023,Atliq_Fusion_Container_Set_of_3,Home Care,Visakhapatnam
423,491ff2,STVJD-1,CAMP_SAN_01,P12,62,30.0,50% OFF,31,42,Sankranti,10-01-2024,16-01-2024,Atliq_Lime_Cool_Bathing_Bar (125GM),Personal Care,Vijayawada
541,6.24E+11,STMLR-0,CAMP_SAN_01,P03,200,206.0,BOGOF,100,541,Sankranti,10-01-2024,16-01-2024,Atliq_Suflower_Oil (1L),Grocery & Staples,Mangalore
703,0f8686,STVSK-0,CAMP_SAN_01,P07,300,24.0,BOGOF,150,92,Sankranti,10-01-2024,16-01-2024,Atliq_Curtains,Home Care,Visakhapatnam


In [13]:
merged.iloc[[40,74]]

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),campaign_name,start_date,end_date,product_name,category,city
40,46d57b,STBLR-9,CAMP_DIW_01,P13,350,94.0,BOGOF,175,329,Diwali,12-11-2023,18-11-2023,Atliq_High_Glo_15W_LED_Bulb,Home Appliances,Bengaluru
74,46d57b,STBLR-9,CAMP_DIW_01,P13,350,94.0,BOGOF,175,329,Diwali,12-11-2023,18-11-2023,Atliq_High_Glo_15W_LED_Bulb,Home Appliances,Bengaluru


In [15]:
merged1=merged.copy()
merged1=merged.drop_duplicates(inplace=False)
merged1.shape

(1500, 15)

### 2. How many cities have more than 5 stores?

In [17]:
merged2=merged1.groupby('city')['store_id'].nunique().reset_index()   # groupby('city') ==> Groups the DataFrame by unique values in the city column. This means all rows belonging to the same city are grouped together for aggregation.
merged2                                                               # After grouping, this selects only the store_id column for further operations (like .count(), .nunique(), etc). It narrows the focus to this particular column within each city group.  
                                                                      # reset.index() --> converts series to DataFrame

,city,store_id
0,Bengaluru,10
1,Chennai,8
2,Coimbatore,5
3,Hyderabad,7
4,Madurai,4
5,Mangalore,3
6,Mysuru,4
7,Trivandrum,2
8,Vijayawada,2
9,Visakhapatnam,5


In [25]:
merged2[merged2.store_id>5]

,city,store_id
0,Bengaluru,10
1,Chennai,8
3,Hyderabad,7


In [26]:
# (merged2>5).sum()  # to be used during pandas series.

In [77]:
# merged2[merged2>5]  # to be used during pandas series

city
Bengaluru    10
Chennai       8
Hyderabad     7
Name: store_id, dtype: int64

### 3. The sales team has identified missing values in the quantity_sold(before_promo) data. **Estimate these values using the median quantity sold before the promotion.** How many missing values were filled, and what is the median used for imputation?

In [28]:
merged1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1500 entries, 0 to 1509
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   event_id                     1500 non-null   object 
 1   store_id                     1500 non-null   object 
 2   campaign_id                  1500 non-null   object 
 3   product_code                 1500 non-null   object 
 4   base_price(before_promo)     1500 non-null   int64  
 5   quantity_sold(before_promo)  1480 non-null   float64
 6   promo_type                   1500 non-null   object 
 7   base_price(after_promo)      1500 non-null   int64  
 8   quantity_sold(after_promo)   1500 non-null   int64  
 9   campaign_name                1500 non-null   object 
 10  start_date                   1500 non-null   object 
 11  end_date                     1500 non-null   object 
 12  product_name                 1500 non-null   object 
 13  category               

In [29]:
merged1.describe()  # in Count row,  "quantity_sold(before_promo)" is missing 20 qtn

,base_price(before_promo),quantity_sold(before_promo),base_price(after_promo),quantity_sold(after_promo)
count,1500.000000,1480.000000,1500.000000,1500.000000
mean,551.966667,139.250000,366.933333,290.315333
std,741.394953,131.725983,600.671902,363.444413
min,50.000000,10.000000,31.000000,9.000000
25%,110.000000,44.500000,67.000000,74.000000
50%,295.000000,78.000000,150.000000,163.000000
75%,860.000000,217.250000,510.000000,337.000000
max,3000.000000,642.000000,2500.000000,2067.000000


In [30]:
merged1.isnull().sum()

event_id                        0
store_id                        0
campaign_id                     0
product_code                    0
base_price(before_promo)        0
quantity_sold(before_promo)    20
promo_type                      0
base_price(after_promo)         0
quantity_sold(after_promo)      0
campaign_name                   0
start_date                      0
end_date                        0
product_name                    0
category                        0
city                            0
dtype: int64

In [31]:
int(merged1['quantity_sold(before_promo)'].median())  # since all qtn, price (before/after are integer type, so changed this to integer type)

78

In [32]:
merged1['quantity_sold(before_promo)'].fillna(merged1['quantity_sold(before_promo)'].median(), inplace=True)

C:\Users\reach\AppData\Local\Temp\ipykernel_2364\1530030982.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged1['quantity_sold(before_promo)'].fillna(merged1['quantity_sold(before_promo)'].median(), inplace=True)
C:\Users\reach\AppData\Local\Temp\ipykernel_2364\1530030982.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged1['qua

In [33]:
merged1.describe()

,base_price(before_promo),quantity_sold(before_promo),base_price(after_promo),quantity_sold(after_promo)
count,1500.000000,1500.000000,1500.000000,1500.000000
mean,551.966667,138.433333,366.933333,290.315333
std,741.394953,131.032861,600.671902,363.444413
min,50.000000,10.000000,31.000000,9.000000
25%,110.000000,45.000000,67.000000,74.000000
50%,295.000000,78.000000,150.000000,163.000000
75%,860.000000,211.000000,510.000000,337.000000
max,3000.000000,642.000000,2500.000000,2067.000000


In [34]:
merged1.isnull().sum()

event_id                       0
store_id                       0
campaign_id                    0
product_code                   0
base_price(before_promo)       0
quantity_sold(before_promo)    0
promo_type                     0
base_price(after_promo)        0
quantity_sold(after_promo)     0
campaign_name                  0
start_date                     0
end_date                       0
product_name                   0
category                       0
city                           0
dtype: int64

### **4.** Identify the **product category** with the **lowest base price before the promotion.**

In [35]:
# merged1['base_price(before_promo)'].min()

In [36]:
# merged1[merged1['base_price(before_promo)']==merged1['base_price(before_promo)'].min()].product_code.unique()

In [37]:
lowest_price_row = merged1.loc[merged1['base_price(before_promo)'].idxmin()]   # Returns the index of the row with the lowest base price
lowest_price_row[['product_code','base_price(before_promo)', 'category']]

product_code                          P10
base_price(before_promo)               50
category                    Personal Care
Name: 12, dtype: object

In [38]:
merged1['base_price(before_promo)'].min()

50

### **5**. What is the **total quantity sold after the promotion** for the **BOGOF promo type** during the **Diwali campaign**?

In [39]:
merged1.head(2)

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),campaign_name,start_date,end_date,product_name,category,city
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327.0,25% OFF,217,287,Diwali,12-11-2023,18-11-2023,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,Chennai
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379.0,BOGOF,185,1622,Sankranti,10-01-2024,16-01-2024,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,Chennai


In [40]:
merged1.promo_type.value_counts()

promo_type
BOGOF           500
25% OFF         400
50% OFF         300
33% OFF         200
500 Cashback    100
Name: count, dtype: int64

In [41]:
# check for dtype and convert to single type
merged1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1500 entries, 0 to 1509
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   event_id                     1500 non-null   object 
 1   store_id                     1500 non-null   object 
 2   campaign_id                  1500 non-null   object 
 3   product_code                 1500 non-null   object 
 4   base_price(before_promo)     1500 non-null   int64  
 5   quantity_sold(before_promo)  1500 non-null   float64
 6   promo_type                   1500 non-null   object 
 7   base_price(after_promo)      1500 non-null   int64  
 8   quantity_sold(after_promo)   1500 non-null   int64  
 9   campaign_name                1500 non-null   object 
 10  start_date                   1500 non-null   object 
 11  end_date                     1500 non-null   object 
 12  product_name                 1500 non-null   object 
 13  category               

In [42]:
import numpy as np
merged1['quantity_sold(before_promo)'] = merged1['quantity_sold(before_promo)'].astype(np.int64)

C:\Users\reach\AppData\Local\Temp\ipykernel_2364\2518265560.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged1['quantity_sold(before_promo)'] = merged1['quantity_sold(before_promo)'].astype(np.int64)


In [43]:
merged1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1500 entries, 0 to 1509
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   event_id                     1500 non-null   object
 1   store_id                     1500 non-null   object
 2   campaign_id                  1500 non-null   object
 3   product_code                 1500 non-null   object
 4   base_price(before_promo)     1500 non-null   int64 
 5   quantity_sold(before_promo)  1500 non-null   int64 
 6   promo_type                   1500 non-null   object
 7   base_price(after_promo)      1500 non-null   int64 
 8   quantity_sold(after_promo)   1500 non-null   int64 
 9   campaign_name                1500 non-null   object
 10  start_date                   1500 non-null   object
 11  end_date                     1500 non-null   object
 12  product_name                 1500 non-null   object
 13  category                     1500 non-

In [44]:
merged5_qtn=merged1[(merged1.campaign_name=="Diwali") & (merged1.promo_type=="BOGOF")]['quantity_sold(after_promo)'].sum()    # Filteres columns with conditions of campaign_name=="Diwali" , promo_type=="BOGOF" 
merged5_qtn

34461

### **6.** Which **store recorded the highest quantity sold after the promotion during the Diwali campaign?**

In [45]:
merged6=merged1[merged1.campaign_name=="Diwali"].groupby('store_id')['quantity_sold(after_promo)'].sum().reset_index()
merged6.sort_values(by='quantity_sold(after_promo)', ascending=False)

# merged6.sort_values(ascending=False)      # it's in pandas series format , without .reset_index() on previous codeline can also convert mereged6 into Pandas DataFrame
# merged6_df = merged6.to_frame(name='total_sold_after_promo')

# merged6_df = merged6.reset_index(name='total_sold_after_promo')  # To use alternative to 
# merged6_df.sort_values(by='total_sold_after_promo', ascending=False)

,store_id,quantity_sold(after_promo)
19,STCHE-4,5013
7,STBLR-7,4893
6,STBLR-6,4857
38,STMYS-1,4779
22,STCHE-7,4779
0,STBLR-0,4759
18,STCHE-3,4605
23,STHYD-0,4460
21,STCHE-6,4445
4,STBLR-4,4408


In [46]:
merged1[merged1.store_id=="STCHE-4"].head(1).city

20    Chennai
Name: city, dtype: object

In [47]:
merged1.loc[merged1.store_id=="STCHE-4",["store_id","city"]].head(1)

,store_id,city
20,STCHE-4,Chennai


### **7.** Understand which campaigns had the most successful outcomes. Compare the **total quantities sold before and after the promotions** for the **Sankranti and Diwali campaigns. Which campaign saw a greater increase in sales?**

In [48]:
merged1.isnull().sum()

event_id                       0
store_id                       0
campaign_id                    0
product_code                   0
base_price(before_promo)       0
quantity_sold(before_promo)    0
promo_type                     0
base_price(after_promo)        0
quantity_sold(after_promo)     0
campaign_name                  0
start_date                     0
end_date                       0
product_name                   0
category                       0
city                           0
dtype: int64

In [49]:
merged1.duplicated().sum()

0

In [50]:
merged7=merged1.groupby('campaign_name')[['quantity_sold(before_promo)', 'quantity_sold(after_promo)']].sum()
merged7

,quantity_sold(before_promo),quantity_sold(after_promo)
campaign_name,,
Diwali,109756,183404
Sankranti,97894,252069


In [51]:
merged7['Sales_increase'] = merged7['quantity_sold(after_promo)']-merged7['quantity_sold(before_promo)']
merged7

,quantity_sold(before_promo),quantity_sold(after_promo),Sales_increase
campaign_name,,,
Diwali,109756,183404,73648
Sankranti,97894,252069,154175


### **8. Which product** recorded the **highest Incremental Revenue Percentage (IR%)** during the **Sankranti campaign**? What is the **IR%** for this product?
Key Metrics:
 - IR% (Incremental Revenue): IR% measures the percentage change in revenue
after a promotion compared to the revenue before the promotion. It helps assess
how effective a promotion was in driving revenue growth.

In [52]:
merged8 = merged1.copy()
merged8.head(2)

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),campaign_name,start_date,end_date,product_name,category,city
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327,25% OFF,217,287,Diwali,12-11-2023,18-11-2023,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,Chennai
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379,BOGOF,185,1622,Sankranti,10-01-2024,16-01-2024,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,Chennai


In [53]:
# Revenue = price*qtn       # calculate for after promo - before promo
merged8['R_before']= merged8['base_price(before_promo)']*merged8['quantity_sold(before_promo)']
merged8['R_after'] = merged8['base_price(after_promo)']*merged['quantity_sold(after_promo)'] 
merged8['IR'] = merged8['R_after']-merged8['R_before']
merged8.head(3)

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),campaign_name,start_date,end_date,product_name,category,city,R_before,R_after,IR
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327,25% OFF,217,287,Diwali,12-11-2023,18-11-2023,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,Chennai,94830,62279.0,-32551.0
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379,BOGOF,185,1622,Sankranti,10-01-2024,16-01-2024,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,Chennai,140230,300070.0,159840.0
2,f30579,STBLR-9,CAMP_DIW_01,P02,860,337,33% OFF,576,488,Diwali,12-11-2023,18-11-2023,Atliq_Sonamasuri_Rice (10KG),Grocery & Staples,Bengaluru,289820,281088.0,-8732.0


In [54]:
merged81=merged8[merged8['campaign_name']=="Sankranti"].groupby('product_code')[['R_before','R_after','IR']].sum()
merged81

,R_before,R_after,IR
product_code,,,
P01,2108376,2011235.0,-97141.0
P02,18374760,17262720.0,-1112040.0
P03,3189600,6118500.0,2928900.0
P04,6813550,12779800.0,5966250.0
P05,59675,35875.0,-23800.0
P06,561080,342722.0,-218358.0
P07,583500,1065450.0,481950.0
P08,2485910,4649330.0,2163420.0
P09,239040,146462.0,-92578.0


In [55]:
# mergedD1=merged8[merged8['campaign_name']=="Diwali"].groupby('product_code')[['R_before','R_after','IR']].sum()
# mergedD1

In [56]:
merged81['IR%'] = round((merged81['IR']*100)/merged81['R_before'],2)
merged81

,R_before,R_after,IR,IR%
product_code,,,,
P01,2108376,2011235.0,-97141.0,-4.61
P02,18374760,17262720.0,-1112040.0,-6.05
P03,3189600,6118500.0,2928900.0,91.83
P04,6813550,12779800.0,5966250.0,87.56
P05,59675,35875.0,-23800.0,-39.88
P06,561080,342722.0,-218358.0,-38.92
P07,583500,1065450.0,481950.0,82.60
P08,2485910,4649330.0,2163420.0,87.03
P09,239040,146462.0,-92578.0,-38.73


In [57]:
merged81.sort_values(by='IR%', ascending=False)

,R_before,R_after,IR,IR%
product_code,,,,
P03,3189600,6118500.0,2928900.0,91.83
P15,16185000,31027500.0,14842500.0,91.71
P13,1740550,3303125.0,1562575.0,89.77
P14,4542060,8534850.0,3992790.0,87.91
P04,6813550,12779800.0,5966250.0,87.56
P08,2485910,4649330.0,2163420.0,87.03
P07,583500,1065450.0,481950.0,82.60
P01,2108376,2011235.0,-97141.0,-4.61
P02,18374760,17262720.0,-1112040.0,-6.05


In [58]:
merged1.loc[merged1.product_code=="P03",["product_name"]].head(1)

,product_name
10,Atliq_Suflower_Oil (1L)


### **9.** Which **store in Visakhapatnam** recorded the **lowest Incremental Sold Units Percentage (ISU%)** during the **Diwali campaign?** What is the **ISU% for that store?**
 - ISU% (Incremental Sold Units): ISU% calculates the percentage change in the
number of units sold after a promotion compared to the units sold before the
promotion. It indicates the effectiveness of a promotion in boosting sales volume.

In [59]:
merged8["ISU"] = merged8["quantity_sold(after_promo)"]-merged8["quantity_sold(before_promo)"]
merged8.head()

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),campaign_name,start_date,end_date,product_name,category,city,R_before,R_after,IR,ISU
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327,25% OFF,217,287,Diwali,12-11-2023,18-11-2023,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,Chennai,94830,62279.0,-32551.0,-40
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379,BOGOF,185,1622,Sankranti,10-01-2024,16-01-2024,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,Chennai,140230,300070.0,159840.0,1243
2,f30579,STBLR-9,CAMP_DIW_01,P02,860,337,33% OFF,576,488,Diwali,12-11-2023,18-11-2023,Atliq_Sonamasuri_Rice (10KG),Grocery & Staples,Bengaluru,289820,281088.0,-8732.0,151
3,4f570c,STBLR-7,CAMP_DIW_01,P05,55,122,25% OFF,41,107,Diwali,12-11-2023,18-11-2023,Atliq_Scrub_Sponge_For_Dishwash,Home Care,Bengaluru,6710,4387.0,-2323.0,-15
4,6d153f,STHYD-5,CAMP_SAN_01,P15,3000,122,500 Cashback,2500,272,Sankranti,10-01-2024,16-01-2024,Atliq_Home_Essential_8_Product_Combo,Combo1,Hyderabad,366000,680000.0,314000.0,150


In [60]:
merged9=merged8[(merged8.campaign_name=="Diwali") & (merged8.city=="Visakhapatnam")].groupby("store_id")[['quantity_sold(before_promo)','quantity_sold(after_promo)','ISU']].sum()
merged9

,quantity_sold(before_promo),quantity_sold(after_promo),ISU
store_id,,,
STVSK-0,1768,3005,1237
STVSK-1,1903,3078,1175
STVSK-2,1701,2860,1159
STVSK-3,1780,2656,876
STVSK-4,1926,2908,982


In [61]:
merged9["ISU%"] = round(merged9["ISU"]*100/merged9["quantity_sold(before_promo)"],2)
merged9.sort_values(by="ISU%", ascending=True)

,quantity_sold(before_promo),quantity_sold(after_promo),ISU,ISU%
store_id,,,,
STVSK-3,1780,2656,876,49.21
STVSK-4,1926,2908,982,50.99
STVSK-1,1903,3078,1175,61.74
STVSK-2,1701,2860,1159,68.14
STVSK-0,1768,3005,1237,69.97


In [62]:
merged8.loc[(merged8.store_id=="STVSK-3")&(merged8.campaign_name=="Diwali"), ["city", "store_id", "campaign_name"]].head(1)

,city,store_id,campaign_name
136,Visakhapatnam,STVSK-3,Diwali


### **10.** Which **promo type** had both a **negative Incremental Revenue Percentage (IR%)** and **Incremental Sold Units Percentage (ISU%)** during the Sankranti campaign?
Key Metrics:
 - IR% (Incremental Revenue): IR% measures the percentage change in revenue
after a promotion compared to the revenue before the promotion. It helps assess
how effective a promotion was in driving revenue growth.
 - ISU% (Incremental Sold Units): ISU% calculates the percentage change in the
number of units sold after a promotion compared to the units sold before the
promotion. It indicates the effectiveness of a promotion in boosting sales volume.

In [63]:
merged10=merged8[merged8.campaign_name=="Sankranti"].groupby("promo_type")[['R_before','IR','quantity_sold(before_promo)','ISU']].sum()
merged10

,R_before,IR,quantity_sold(before_promo),ISU
promo_type,,,,
25% OFF,935195,-367808.0,6601,-1294
33% OFF,20483136,-1209181.0,33624,13835
50% OFF,527482,-163918.0,4451,1649
500 Cashback,16185000,14842500.0,5395,7016
BOGOF,19355170,17095885.0,47823,132969


In [64]:
merged10["IR%"] = round((merged10["IR"]*100)/merged10["R_before"],2)
merged10["ISU%"]=round((merged10["ISU"]*100)/merged10["quantity_sold(before_promo)"],2)

In [330]:
merged10

,R_before,IR,quantity_sold(before_promo),ISU,IR%,ISU%
promo_type,,,,,,
25% OFF,935195,-367808,6601,-1294,-39.33,-19.60
33% OFF,20483136,-1209181,33624,13835,-5.90,41.15
50% OFF,527482,-163918,4451,1649,-31.08,37.05
500 Cashback,16185000,14842500,5395,7016,91.71,130.05
BOGOF,19355170,17095885,47823,132969,88.33,278.04


In [65]:
merged10.sort_values(by="ISU%", ascending=False)

,R_before,IR,quantity_sold(before_promo),ISU,IR%,ISU%
promo_type,,,,,,
BOGOF,19355170,17095885.0,47823,132969,88.33,278.04
500 Cashback,16185000,14842500.0,5395,7016,91.71,130.05
33% OFF,20483136,-1209181.0,33624,13835,-5.90,41.15
50% OFF,527482,-163918.0,4451,1649,-31.08,37.05
25% OFF,935195,-367808.0,6601,-1294,-39.33,-19.60


In [66]:
merged11=merged10[(merged10['IR%']<0) & (merged10['ISU%']<0)]
merged11

,R_before,IR,quantity_sold(before_promo),ISU,IR%,ISU%
promo_type,,,,,,
25% OFF,935195,-367808.0,6601,-1294,-39.33,-19.6


In [82]:
# by Filtering capaign_id instead of campaign_name

In [71]:
merged8[merged8.campaign_id=="CAMP_SAN_01"].shape

(750, 19)

In [70]:
# merged8[merged8.campaign_name=="Sankranti"].shape

(750, 19)

In [74]:
merged10A=merged8[merged8.campaign_id=="CAMP_SAN_01"].groupby('promo_type')[['R_before','IR','quantity_sold(before_promo)','ISU']].sum()
merged10A

,R_before,IR,quantity_sold(before_promo),ISU
promo_type,,,,
25% OFF,935195,-367808.0,6601,-1294
33% OFF,20483136,-1209181.0,33624,13835
50% OFF,527482,-163918.0,4451,1649
500 Cashback,16185000,14842500.0,5395,7016
BOGOF,19355170,17095885.0,47823,132969


In [81]:
merged10A['IR%']=round((merged10A["IR"]*100)/merged10A["R_before"],2)
merged10A["ISU%"]=round((merged10A["ISU"]*100)/merged10A["quantity_sold(before_promo)"],2)
merged10A

,R_before,IR,quantity_sold(before_promo),ISU,IR%,ISU%
promo_type,,,,,,
25% OFF,935195,-367808.0,6601,-1294,-39.33,-19.60
33% OFF,20483136,-1209181.0,33624,13835,-5.90,41.15
50% OFF,527482,-163918.0,4451,1649,-31.08,37.05
500 Cashback,16185000,14842500.0,5395,7016,91.71,130.05
BOGOF,19355170,17095885.0,47823,132969,88.33,278.04


In [128]:
df10=merged.copy()
df10.head(3)

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),campaign_name,start_date,end_date,product_name,category,city
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327.0,25% OFF,217,287,Diwali,12-11-2023,18-11-2023,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,Chennai
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379.0,BOGOF,185,1622,Sankranti,10-01-2024,16-01-2024,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,Chennai
2,f30579,STBLR-9,CAMP_DIW_01,P02,860,337.0,33% OFF,576,488,Diwali,12-11-2023,18-11-2023,Atliq_Sonamasuri_Rice (10KG),Grocery & Staples,Bengaluru


In [129]:
df10.duplicated().sum()

10

In [130]:
df10.drop_duplicates(inplace=True)
df10.shape

(1500, 15)

In [131]:
df10.isnull().sum()

event_id                        0
store_id                        0
campaign_id                     0
product_code                    0
base_price(before_promo)        0
quantity_sold(before_promo)    20
promo_type                      0
base_price(after_promo)         0
quantity_sold(after_promo)      0
campaign_name                   0
start_date                      0
end_date                        0
product_name                    0
category                        0
city                            0
dtype: int64

In [132]:
df10["quantity_sold(before_promo)"].median()

78.0

In [133]:
df10.fillna(df10["quantity_sold(before_promo)"].median(), inplace=True)

In [134]:
df10.isnull().sum()

event_id                       0
store_id                       0
campaign_id                    0
product_code                   0
base_price(before_promo)       0
quantity_sold(before_promo)    0
promo_type                     0
base_price(after_promo)        0
quantity_sold(after_promo)     0
campaign_name                  0
start_date                     0
end_date                       0
product_name                   0
category                       0
city                           0
dtype: int64

In [135]:
df10.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1500 entries, 0 to 1509
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   event_id                     1500 non-null   object 
 1   store_id                     1500 non-null   object 
 2   campaign_id                  1500 non-null   object 
 3   product_code                 1500 non-null   object 
 4   base_price(before_promo)     1500 non-null   int64  
 5   quantity_sold(before_promo)  1500 non-null   float64
 6   promo_type                   1500 non-null   object 
 7   base_price(after_promo)      1500 non-null   int64  
 8   quantity_sold(after_promo)   1500 non-null   int64  
 9   campaign_name                1500 non-null   object 
 10  start_date                   1500 non-null   object 
 11  end_date                     1500 non-null   object 
 12  product_name                 1500 non-null   object 
 13  category               

In [136]:
# Convert the data type of quantity_sold(before_promo)  1510 non-null   float64
# import numpy as np
# df10["quantity_sold(before_promo)"]=df10["quantity_sold(before_promo)"].astype(np.int64)

In [137]:
df10.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1500 entries, 0 to 1509
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   event_id                     1500 non-null   object 
 1   store_id                     1500 non-null   object 
 2   campaign_id                  1500 non-null   object 
 3   product_code                 1500 non-null   object 
 4   base_price(before_promo)     1500 non-null   int64  
 5   quantity_sold(before_promo)  1500 non-null   float64
 6   promo_type                   1500 non-null   object 
 7   base_price(after_promo)      1500 non-null   int64  
 8   quantity_sold(after_promo)   1500 non-null   int64  
 9   campaign_name                1500 non-null   object 
 10  start_date                   1500 non-null   object 
 11  end_date                     1500 non-null   object 
 12  product_name                 1500 non-null   object 
 13  category               

In [138]:
df10.head(2)

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),campaign_name,start_date,end_date,product_name,category,city
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327.0,25% OFF,217,287,Diwali,12-11-2023,18-11-2023,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,Chennai
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379.0,BOGOF,185,1622,Sankranti,10-01-2024,16-01-2024,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,Chennai


In [139]:
df10["R_before"] = df10["base_price(before_promo)"]*df10["quantity_sold(before_promo)"]
df10["R_after"]=df10["base_price(after_promo)"]*df10["quantity_sold(after_promo)"]
df10["IR"] = df10["R_after"]-df10["R_before"]
df10["ISU"]=df10["quantity_sold(after_promo)"]-df10["quantity_sold(before_promo)"]
df10.head(3)

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),campaign_name,start_date,end_date,product_name,category,city,R_before,R_after,IR,ISU
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327.0,25% OFF,217,287,Diwali,12-11-2023,18-11-2023,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,Chennai,94830.0,62279,-32551.0,-40.0
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379.0,BOGOF,185,1622,Sankranti,10-01-2024,16-01-2024,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,Chennai,140230.0,300070,159840.0,1243.0
2,f30579,STBLR-9,CAMP_DIW_01,P02,860,337.0,33% OFF,576,488,Diwali,12-11-2023,18-11-2023,Atliq_Sonamasuri_Rice (10KG),Grocery & Staples,Bengaluru,289820.0,281088,-8732.0,151.0


In [140]:
df10_per=df10[df10.campaign_name=="Sankranti"].groupby('promo_type')[["quantity_sold(before_promo)","quantity_sold(after_promo)","ISU","R_before","R_after","IR"]].sum()
df10_per

,quantity_sold(before_promo),quantity_sold(after_promo),ISU,R_before,R_after,IR
promo_type,,,,,,
25% OFF,6601.0,5307,-1294.0,935195.0,567387,-367808.0
33% OFF,33624.0,47459,13835.0,20483136.0,19273955,-1209181.0
50% OFF,4451.0,6100,1649.0,527482.0,363564,-163918.0
500 Cashback,5395.0,12411,7016.0,16185000.0,31027500,14842500.0
BOGOF,47823.0,180792,132969.0,19355170.0,36451055,17095885.0


In [141]:
df10_per["ISU%"]=round((df10_per["ISU"]/df10_per["quantity_sold(before_promo)"])*100,2)
df10_per["IR%"]=round((df10_per["IR"]/df10_per["R_before"])*100,2)

In [142]:
df10_per

,quantity_sold(before_promo),quantity_sold(after_promo),ISU,R_before,R_after,IR,ISU%,IR%
promo_type,,,,,,,,
25% OFF,6601.0,5307,-1294.0,935195.0,567387,-367808.0,-19.60,-39.33
33% OFF,33624.0,47459,13835.0,20483136.0,19273955,-1209181.0,41.15,-5.90
50% OFF,4451.0,6100,1649.0,527482.0,363564,-163918.0,37.05,-31.08
500 Cashback,5395.0,12411,7016.0,16185000.0,31027500,14842500.0,130.05,91.71
BOGOF,47823.0,180792,132969.0,19355170.0,36451055,17095885.0,278.04,88.33


In [143]:
df10_per[(df10_per["IR%"]<0) & (df10_per["ISU"]<0)]

,quantity_sold(before_promo),quantity_sold(after_promo),ISU,R_before,R_after,IR,ISU%,IR%
promo_type,,,,,,,,
25% OFF,6601.0,5307,-1294.0,935195.0,567387,-367808.0,-19.6,-39.33
